In [2]:
# [0] Python実行環境確認
import sys
print(sys.executable)

c:\Users\lapis\Desktop\LoL_WorkSp_win\pyLoL-_WorkSp\pyLoL-v2\.venv\Scripts\python.exe


In [3]:
# [1] 初期設定・APIキー読み込み
import os
from dotenv import load_dotenv

from autoLeague.dataset.generator import DataGenerator
from autoLeague.utils.csv_utils import save_to_csv
from autoLeague.utils.patch_info import get_patch_info

# Load API Key from RiotAPI.env
load_dotenv("../RiotAPI.env")
API_KEY = os.environ.get("API_KEY")
if not API_KEY:
    raise ValueError("API_KEY not found in RiotAPI.env. Please check the file.")

print(f"API_KEY: {API_KEY[:15]}...{API_KEY[-5:]}")  # Partially masked for security
get_patch_info()

API_KEY: RGAPI-9f2974f5-...911df
Current time: 2026-01-08 03:26:57
Most recent past patch version: 25.24
Patch date: 2025-12-10


<div style="font-size: 13px">
  <b>✅ Output example</b><br><br>
  <b>Current time</b> : <code>2025-07-06 01:29:48</code><br>
  <b>Most recent past patch</b> : <code>25.13</code><br>
  <b>Patch date</b> : <code>2025-06-25</code>

  <hr>

  <b>📝 Note</b><br><br>
  - Replay <code>.rofl</code> files can only be downloaded <b>for the current patch version</b>, so check the <b>patch schedule</b> first and enter the date in the variable below.<br>
  - Any date between the latest patch date and today will work.


<pre><code>patch_start_datetime = "2025.06.30"  # → downloads matches from 2025-06-30 up to today
</code></pre>

</div>

In [4]:
# [2] DataGenerator初期化
dg = DataGenerator(api_key=API_KEY, count=100)

In [4]:
# [3] OPTION-1: 単一ティアのmatchId取得
matchIds_challenger = dg.get_tier_matchIds(
                                            queue='RANKED_SOLO_5x5',         # キュータイプ: サモナーズリフト ランク戦 ソロ5v5
                                            tier='CHALLENGER',               # ティア: CHALLENGER (最高競技ティア)
                                            division='I',                    # ディビジョン: I (CHALLENGER〜MASTERは技術的に"I"のみ)
                                            max_ids=100,                     # 取得するマッチIDの最大数
                                            # patch_start_datetime='2025.06.26',  # このパッチリリース日以降の試合のみ取得
                                            patch_start_datetime='2026.1.1',  # このパッチリリース日以降の試合のみ取得
                                            min_game_duration=15,           # 最小ゲーム時間（分）- 早期サレンダー/リメイクを除外
                                            max_game_duration=45,            # 最大ゲーム時間（分）- 異常に長い試合を除外
                                                                            # この設定では、6月26日以降のKRサーバーのCHALLENGERティアのソロランク戦が収集されます
                                                                            # 15〜30分の試合に限定されます
                                                                            # 最大サモナー数は300に設定（CHALLENGERは最高ティアでプレイヤー数が少ないため妥当な数）
                                           max_workers=16
                                           )

save_to_csv(matchIds_challenger, r'../matchids/sample_challenger_patch25_13_1.csv', 'matchid')

[INFO] get_puuids: CHALLENGER-I から 50 人のPUUIDを取得しました


[CHALLENGER-I] fetch matchIds: 100%|====================================================| 50/50 [18:53<00:00, 22.66s/it]

[INFO] CHALLENGER-I: 合計 354 件のユニークなmatchIdを取得しました


In [6]:
import pandas as pd
import shutil
from pathlib import Path
from datetime import datetime

src = Path(r"c:\Users\lapis\Desktop\LoL_WorkSp_win\pyLoL-_WorkSp\pyLoL-v2\matchids\sample_challenger_patch25_13_1.csv")
if not src.exists():
    raise FileNotFoundError(f"{src} not found")

df = pd.read_csv(src)

# ensure there's a 'matchid' column (try common alternatives)
if "matchid" not in df.columns:
    cols_lower = {c.lower(): c for c in df.columns}
    if "matchid" in cols_lower:
        df.rename(columns={cols_lower["matchid"]: "matchid"}, inplace=True)
    elif len(df.columns) == 1:
        df.columns = ["matchid"]
    else:
        raise ValueError("No 'matchid' column found in CSV")

before = len(df)

# normalize and remove empty/null matchids
df = df[df["matchid"].notna()].copy()
df["matchid"] = df["matchid"].astype(str).str.strip()
df = df[df["matchid"] != ""]

# drop duplicate matchids (keep first occurrence)
df = df.drop_duplicates(subset="matchid", keep="first").reset_index(drop=True)
after = len(df)

# backup original file and overwrite with cleaned file
bak = src.with_suffix(src.suffix + f".bak_{datetime.now().strftime('%Y%m%d_%H%M%S')}")
shutil.copy2(src, bak)
df.to_csv(src, index=False)

print(f"{src.name}: before={before}, after={after}, removed={before - after}")
print(f"Backup saved to: {bak}")

sample_challenger_patch25_13_1.csv: before=580, after=354, removed=226
Backup saved to: c:\Users\lapis\Desktop\LoL_WorkSp_win\pyLoL-_WorkSp\pyLoL-v2\matchids\sample_challenger_patch25_13_1.csv.bak_20260108_032957


In [ ]:
# [4] OPTION-2: 複数ティアのmatchId一括取得
# ── 設定値を上で一度定義 ──────────────────────────────
PATCH_START     = "2026.1.1"                              # patch_start_datetime の値
PATCH_VER       = "25_24"                                   # ファイル名に入れるパッチバージョン
TIERS           = ["CHALLENGER", "GRANDMASTER", "MASTER"]   # ["DIAMOND", "EMERALD", "PLATINUM", "GOLD", "SILVER", "BRONZE", "IRON"]
MAXIDS_PER_TIER = [300, 700, 1000]                          # 各ティアごとに最大何個のmatchidを取得するか
DIVISIONS       = ["I"]                                     # ダイヤモンド以下でのみ I 〜 IV 適用 ["I", "II", "III", "IV"]
OUT_DIR         = "matchids"                                # 保存先パス (例: pyLoL/matchids)

# ── フォルダ準備 ───────────────────────────────────────────────
import os, pathlib
pathlib.Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# ── ダウンロード + CSV保存ループ ────────────────────────────
for tier, max_ids in zip(TIERS, MAXIDS_PER_TIER):
    for div in DIVISIONS:
        match_ids = dg.get_tier_matchIds(
            queue="RANKED_SOLO_5x5",
            tier=tier,
            division=div,
            max_ids=max_ids,
            patch_start_datetime=PATCH_START,
            min_game_duration=15,
            max_game_duration=60,
            max_workers=8
        )
        save_to_csv(
            match_ids,
            rf"{OUT_DIR}/matchids_{tier}_{div}_patch{PATCH_VER}.csv",
            "matchid",
        )